In [2]:
!pip install sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.6/615.6 kB 24.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import requests
import gzip
import io
import psycopg2
import csv




def load_data(filename: str, color):
    release_api_url = f"https://api.github.com/repos/DataTalksClub/nyc-tlc-data/releases/tags/{color}"
    response = requests.get(release_api_url)
    response.raise_for_status()  # Raise error for failed requests
    release_info = response.json()

    # Step 2: Find the asset named "green_tripdata_2019-01.csv.gz"
    asset_url = None
    for asset in release_info.get('assets', []):
        if asset.get('name') == filename:
            asset_url = asset.get('browser_download_url')
            break

    if not asset_url:
        raise Exception(f"Asset {filename} not found in the release.")

    print(f"Downloading {asset_url}...")
    file_response = requests.get(asset_url)
    file_response.raise_for_status()

    # Step 3: Decompress the downloaded gzip file to get CSV content
    with gzip.open(io.BytesIO(file_response.content), 'rt') as f:
        csv_data = f.read()

    # Step 4: Connect to the PostgreSQL database on localhost:5432 (database "test")
    conn = psycopg2.connect(
        dbname="test",
        user="test",  # adjust username as needed
        password="test",
        host="localhost",
        port=5432
    )
    cursor = conn.cursor()

    # Step 5: Create the table "green_taxies" if it doesn't exist
    # Use Python's csv module to read the header row from the CSV data
    csv_header_io = io.StringIO(csv_data)
    reader = csv.reader(csv_header_io)
    header = next(reader)  # Extract header row as list of column names

    # Build a CREATE TABLE statement with each column as TEXT.
    # Note: In production you might want to define more specific types.
    columns_sql = ", ".join([f'"{col}" TEXT' for col in header])
    if color == "green":
        create_table_sql = f"""

        CREATE TABLE if not exists public.green_taxies (
            "VendorID" int4 NULL,
            lpep_pickup_datetime timestamp NULL,
            lpep_dropoff_datetime timestamp NULL,
            store_and_fwd_flag varchar(50) NULL,
            "RatecodeID" text NULL,
            "PULocationID" int4 NULL,
            "DOLocationID" text NULL,
            passenger_count int4 NULL,
            trip_distance float4 NULL,
            fare_amount float4 NULL,
            extra float4 NULL,
            mta_tax float4 NULL,
            tip_amount float4 NULL,
            tolls_amount float4 NULL,
            ehail_fee float(50) NULL,
            improvement_surcharge float4 NULL,
            total_amount float4 NULL,
            payment_type float4 NULL,
            trip_type float4 NULL,
            congestion_surcharge varchar(50) NULL
        );
        """
    elif color == "fhv":
        create_table_sql = """
        CREATE TABLE if not exists public.fhv_taxies (
            dispatching_base_num varchar(50) NULL,
            pickup_datetime timestamp NULL,
            "dropOff_datetime" timestamp NULL,
            "PUlocationID" int NULL,
            "DOlocationID" int NULL,
            "SR_Flag" varchar(50) NULL,
            "Affiliated_base_number" varchar(50) NULL
        );
        """
    else:
        create_table_sql = """
        CREATE TABLE if not exists public.yellow_taxies (
            "VendorID" int NULL,
            tpep_pickup_datetime timestamp NULL,
            tpep_dropoff_datetime timestamp NULL,
            passenger_count int NULL,
            trip_distance float NULL,
            "RatecodeID" int NULL,
            store_and_fwd_flag varchar(50) NULL,
            "PULocationID" int NULL,
            "DOLocationID" int NULL,
            payment_type int NULL,
            fare_amount float NULL,
            extra float NULL,
            mta_tax float NULL,
            tip_amount float NULL,
            tolls_amount float NULL,
            improvement_surcharge float NULL,
            total_amount float NULL,
            congestion_surcharge varchar(50) NULL
        );
        """
        
    cursor.execute(create_table_sql)
    conn.commit()

    print(f"Table {color}_taxies created (or already exists).")

    # Step 6: Load CSV data into the table using PostgreSQL's COPY command
    print(len(csv_data))
    csv_copy_io = io.StringIO(csv_data)
    
    copy_sql = f"COPY {color}_taxies FROM STDIN WITH CSV HEADER"
    cursor.copy_expert(copy_sql, csv_copy_io)

    conn.commit()
    cursor.close()
    conn.close()

    print("Data loaded successfully into the green_taxies table!")

for color in ["fhv"]:
    for year in [2019, 2020]:
        for month in ["0" + str(x) if len(str(x)) == 1 else str(x) for x in list(range(1,13))]:
            load_data(f"{color}_tripdata_{year}-{month}.csv.gz", color)
#                         https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-01.csv.gz

Table fhv_taxies created (or already exists).
1428503544
Data loaded successfully into the green_taxies table!
Table fhv_taxies created (or already exists).
108225919
Data loaded successfully into the green_taxies table!
Table fhv_taxies created (or already exists).
93980700
Data loaded successfully into the green_taxies table!
Table fhv_taxies created (or already exists).
123008542
Data loaded successfully into the green_taxies table!
Table fhv_taxies created (or already exists).
131387979
Data loaded successfully into the green_taxies table!
Table fhv_taxies created (or already exists).
127591175
Data loaded successfully into the green_taxies table!
Table fhv_taxies created (or already exists).
123323044
Data loaded successfully into the green_taxies table!
Table fhv_taxies created (or already exists).
119173810
Data loaded successfully into the green_taxies table!
Table fhv_taxies created (or already exists).
78714715
Data loaded successfully into the green_taxies table!
Table fhv_t

BadCopyFileFormat: missing data for column "pickup_datetime"
CONTEXT:  COPY fhv_taxies, line 101142: "B02765"


In [1]:
st = "1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95,".split(sep=",")
cols = "VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge".split(sep=",")
len(st), len(cols)

(18, 18)

#### Analytics questions

Question 5. Taxi Quarterly Revenue Growth